In [ ]:
# pacotes

import requests
import pandas as pd
import json as json
import psycopg2
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

In [ ]:
# Caminho para o .env da EUROLASER (Railway)

# Depois essa pasta vamos trocar pela pasta que ficará no servidor da EUROLASER
dotenv_path = "URL"

# Carrega as variáveis do arquivo .env, forçando substituição se já houver algo na memória
load_dotenv(dotenv_path, override=True)

# Recupera variáveis de ambiente
host = os.getenv("host")
porta = os.getenv("porta")
usuario = os.getenv("usuario")
senha = quote_plus(os.getenv("senha"))  # Protege caracteres especiais
banco = os.getenv("database")

# Mostra para confirmação
print("Conectando em:", host, porta, usuario, banco)

# Cria a engine
engine = create_engine(f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}")

In [ ]:
# Carrega o arquivo com as credenciais

with open('CREDENCIAIS') as f:
    config = json.load(f)

token = config['TOKEN_API']

In [ ]:
# API OP

pagina_op = 1
todos_dados_op = []

# Loop de paginação
while True:
    url_op = f"API_OP"
    headers_op = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_op, headers=headers_op)
    
    if response.status_code == 200:
        dados = response.json()
        
        # Se a resposta vier vazia, terminamos a coleta
        if not dados:
            break

        todos_dados_op.extend(dados)
        print(f"✅ Página {pagina_op} carregada. Total acumulado: {len(todos_dados_op)}")
        pagina_op += 1
    else:
        print(f"❌ Erro na página {pagina_op}: {response.status_code} - {response.text}")
        break

# Converter em DataFrame
df_ops = pd.DataFrame(todos_dados_op)

# Visualizar
print("🔍 Primeiras ordens de produção:")
print(df_ops.head())

In [ ]:
# API PEDIDOS

pagina_pedidos = 1
todos_dados_pedidos = []
limite_paginas = 1000000  

while True:
    url_pedidos = f"API/PEDIDOS"

    headers_pedidos = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_pedidos, headers=headers_pedidos)
    
    if response.status_code == 200:
        dados_pedidos = response.json()
        
        if not dados_pedidos:
            break

        todos_dados_pedidos.extend(dados_pedidos)
        print(f"✅ Página {pagina_pedidos} carregada. Total acumulado: {len(todos_dados_pedidos)}")

        pagina_pedidos += 1

        # Interrompe
        if pagina_pedidos > limite_paginas:
            print("🚧 Limite de páginas atingido (teste)")
            break

    else:
        print(f"❌ Erro na página {pagina_pedidos}: {response.status_code} - {response.text}")
        break

# Converte em DataFrame
df_pedidos = pd.DataFrame(todos_dados_pedidos)
print(df_pedidos.head())


In [ ]:
# Consumindo a API de Clientes

pagina = 1
todos_dados = []

# Loop de paginação
while True:
    url = f"API/CLIENTES"
    headers = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        dados = response.json()
        
        # Se a resposta vier vazia, terminamos a coleta
        if not dados:
            break

        todos_dados.extend(dados)
        print(f"✅ Página {pagina} carregada. Total acumulado: {len(todos_dados)}")
        pagina += 1
    else:
        print(f"❌ Erro na página {pagina}: {response.status_code} - {response.text}")
        break

# Converter em DataFrame
df_clientes = pd.DataFrame(todos_dados)

# Visualizar
print("🔍 Primeiras ordens de produção:")
print(df_clientes.head())

In [ ]:
# verificando registros 

df_clientes.head()

In [ ]:
# seleciona as tabelas importantes

df_clientes  = df_clientes [['nome','fantasia','telefone','email','nome_cid','cod_est','ativo','codcli']]
df_clientes.head()

Renomear as colunas que ja existem para nao precisar trocar nas rotas/apis/front etc...

1. id
2. codigo
3. nome
4. nomefantasia

In [ ]:
# Rename

df_clientes.rename(columns={
    'codcli': 'codigo',
    'fantasia': 'nomeFantasia'
}, inplace=True)

In [ ]:
df_clientes.head()

Precisamos criar um processo, que verifique se todos os clientes na tabela de ordens de produção estão na tabela de clientes, se houverem clientes que não estão, precisamos inserir antes de salvar 

In [ ]:
df_pedidos.head(1)

In [ ]:
# lista de clientes 

lista_clientes = df_clientes['codigo'].tolist()

# Verificando se esses clientes da lista de OP estão no dataframe de OP
df_ops ['flagvalidacao'] = df_ops['codcli'].isin(lista_clientes).astype(int)
df_pedidos ['flagvalidacao'] = df_pedidos ['codcli'].isin(lista_clientes).astype(int)

In [ ]:
# verificando

df_ops['flagvalidacao'].value_counts()

In [ ]:
# verificando

df_pedidos['flagvalidacao'].value_counts()

In [ ]:
# verificando as ops
df_ops_nao_identificados = df_ops.query("flagvalidacao == 0")
qtd_ordensnao_identificados = df_ops_nao_identificados ['numero'].nunique()
qtd_clientesnao_identificados = df_ops_nao_identificados ['codcli'].nunique()


# verificando os pedidos
df_pedidos_nao_identificados = df_pedidos.query("flagvalidacao == 0")
qtd_pedidosnao_identificados = df_pedidos_nao_identificados ['numero'].nunique()
qtd_clientesnao_identificados_pedidos = df_pedidos_nao_identificados ['codcli'].nunique()


print("Quantidade de ordens com clientes não identificados ", qtd_ordensnao_identificados)
print("Quantidade de clientes não identificados: ", qtd_clientesnao_identificados, "\n")

print("Quantidade de Pedidos com clientes não identificados ", qtd_pedidosnao_identificados)
print("Quantidade de clientes não identificados: ", qtd_clientesnao_identificados_pedidos)

In [ ]:
# criando uma lista, somente com os clientes que nao estao na tabela de clientes

lista_clientes_nao_identificados = df_ops_nao_identificados ['codcli'].drop_duplicates().tolist()
lista_clientes_nao_identificados_pedidos = df_pedidos_nao_identificados['codcli'].drop_duplicates().tolist()

In [ ]:
# exibindo a lista

lista_clientes_nao_identificados

In [ ]:
# exibindo a lista

lista_clientes_nao_identificados_pedidos

In [ ]:
# unindo as duas listas

lista_clientes_nao_identificados = list(
    set(lista_clientes_nao_identificados + lista_clientes_nao_identificados_pedidos)
)


In [ ]:
df_clientes.head()

In [ ]:
# Pega as colunas do DataFrame original

colunas = df_clientes.columns.tolist()

# Cria uma lista de dicionários com os novos clientes

dados_novos_clientes = []
for codcli in lista_clientes_nao_identificados:
    cliente_dict = {col: ("Não Identificado" if col != "codigo" else codcli) for col in colunas}
    dados_novos_clientes.append(cliente_dict)

# Cria o novo DataFrame com os registros ausentes

novos_clientes = pd.DataFrame(dados_novos_clientes)

# Garante a mesma ordem de colunas

novos_clientes = novos_clientes[df_clientes.columns]

# Concatena com o df original

df_clientes = pd.concat([df_clientes, novos_clientes], ignore_index=True)

print(f"✅ Clientes adicionados: {len(novos_clientes)}")

In [ ]:
# Reflete a estrutura do banco para capturar a tabela já existen
metadata = MetaData()
metadata.reflect(bind=engine)
tabela_cliente = metadata.tables["Cliente"]  # Tabela com nome e capitalização exatos

# Converte DataFrame em lista de dicionários
dados = df_clientes.to_dict(orient="records")

# Cria comando de insert com fallback se houver conflito de PK
stmt = insert(tabela_cliente).values(dados)
stmt = stmt.on_conflict_do_nothing(index_elements=["codigo"])  # PK = codigo

# Executa a operação no banco
with engine.begin() as conn:
    conn.execute(stmt)
